In [1]:
import os
import sys
newPath = os.path.dirname(os.path.dirname(os.path.abspath("")))
if newPath not in sys.path:
    sys.path.append(newPath)

from main import *
import networkx as nx

m=bi()

c:\Users\SSosa\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


jax.local_device_count 16


## Data

In [215]:
network = jnp.array([
    [0, 3,0, 0, 1],
    [3, 0, 8, 0, 2],
    [0, 8, 0, 1, 0],
    [0, 0, 1, 0, 4],
    [1, 2, 0, 4, 0]
])
G = nx.from_numpy_array(network, create_using=nx.DiGraph)

In [3]:
bi.net.is_symmetric(network)

Array(True, dtype=bool)

# Nodal measures

## Degree

In [4]:
print(m.net.indegree(network))
print(G.in_degree())

[2 3 2 2 3]
[(0, 2), (1, 3), (2, 2), (3, 2), (4, 3)]


In [5]:
print(m.net.outdegree(network))
print(print(G.out_degree()))

[2 3 2 2 3]
[(0, 2), (1, 3), (2, 2), (3, 2), (4, 3)]
None


In [6]:
print(m.net.degree(network))
print(print(G.degree()))

[4 6 4 4 6]
[(0, 4), (1, 6), (2, 4), (3, 4), (4, 6)]
None


In [7]:
# n alters to do 

## Strength


In [8]:
print(m.net.strength(network))
print(G.degree(weight='weight'))

[ 8 26 18 10 14]
[(0, 8), (1, 26), (2, 18), (3, 10), (4, 14)]


In [9]:
print(m.net.instrength(network))
print(G.in_degree(weight='weight'))

[ 4 13  9  5  7]
[(0, 4), (1, 13), (2, 9), (3, 5), (4, 7)]


In [10]:
print(m.net.outstrength(network))
print(G.out_degree(weight='weight'))

[ 4 13  9  5  7]
[(0, 4), (1, 13), (2, 9), (3, 5), (4, 7)]


## Eingenvector

In [11]:
print(bi.net.eigenvector(network))
nx.eigenvector_centrality(G, max_iter=300, weight='weight')

[0.25136444 0.67883556 0.61531811 0.17952819 0.25524783]


{0: 0.25138262091453806,
 1: 0.6787756660010696,
 2: 0.6153735144830952,
 3: 0.17951397648549727,
 4: 0.25526563236368144}

## Clustering coefficient

In [12]:
print(bi.net.cc(network))
nx.clustering(G).values()

[1.         0.33333333 0.         0.         0.33333333]


dict_values([1.0, 0.3333333333333333, 0, 0, 0.3333333333333333])

## Betweeness

In [16]:
    def single_source_dijkstra(src):
        # Initialize distances and visited status
        dist = jnp.full((n_nodes,), jnp.inf)
        dist = dist.at[src].set(0)
        visited = jnp.zeros((n_nodes,), dtype=bool)

        def relax_step(carry, _):
            dist, visited = carry
            # Find the closest unvisited node
            unvisited_dist = jnp.where(visited, jnp.inf, dist)
            u = jnp.argmin(unvisited_dist)
            visited = visited.at[u].set(True)
            # Relax distances for neighbors of the selected node
            new_dist = jnp.where(
                ~visited,
                jnp.minimum(dist, dist[u] + network[u]),
                dist
            )
            return (new_dist, visited), None

        (dist, _), _ = jax.lax.scan(relax_step, (dist, visited), None, length=n_nodes)

        return dist

In [216]:
nx.betweenness_centrality(G)

{0: 0.0, 1: 0.25, 2: 0.08333333333333333, 3: 0.08333333333333333, 4: 0.25}

In [120]:
network=jnp.array(network)
network=network.at[jnp.where(network == 0)].set(jnp.inf)
n_nodes=network.shape[0]
# Number of shortest paths from s
res =single_source_dijkstra(0)
res
# Accumulate the contribution

c:\Users\SSosa\AppData\Local\Programs\Python\Python312\Lib\site-packages\jax\_src\ops\scatter.py:92: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float64 to dtype=int64 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(


Array([0., 3., 6., 5., 1.], dtype=float64, weak_type=True)

In [219]:
network = jnp.array([
    [0, 3,0, 0, 1],
    [3, 0, 8, 0, 2],
    [0, 8, 0, 1, 0],
    [0, 0, 1, 0, 4],
    [1, 2, 0, 4, 0]
])
G = nx.from_numpy_array(network, create_using=nx.DiGraph)

network

Array([[0, 3, 0, 0, 1],
       [3, 0, 8, 0, 2],
       [0, 8, 0, 1, 0],
       [0, 0, 1, 0, 4],
       [1, 2, 0, 4, 0]], dtype=int64)

In [257]:
s=jnp.argsort(geo[0,:])
i=0 # je part de
j=1 #je vais a
# pour chaque noeud y a t il un chemin dont la distance est equivalente
x=4

def test_equivalent_path(path_ij,i,j,x):
    path_ij_by_k=geo[i,x]+geo[x,j]
    return path_ij==path_ij_by_k

def all_alters_equivalent_path(network,geo,i,j): 
    path_ij=geo[i,j] 
    r=[i]    
    for a in range(network.shape[0]):
        if a != i and a != j:
            if geo[i,a] > path_ij:
                continue

            if test_equivalent_path(path_ij,i,j,a):
                r.append(a)
    return r  

def accumulate(s,p,sig,i,betweenness):

    ss=list(np.array(list(s)))
    delta = dict.fromkeys(ss, 0)
    while ss:
        w = ss.pop()
        coeff = (1 + delta[w]) / sig

        for v in p:
            delta[v] += sig * coeff

        if w != i:
            betweenness[w] += delta[w]

    return betweenness, delta

geo = bi.net.geodesic_distance(network)
s=jnp.argsort(geo[i,:])
p=all_alters_equivalent_path(network,geo,i,j)     
sig=len(p)*2
betweenness=np.zeros(len(s))
accumulate(s,p,sig,i,betweenness)

c:\Users\SSosa\AppData\Local\Programs\Python\Python312\Lib\site-packages\jax\_src\ops\scatter.py:92: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float64 to dtype=int64 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(


(array([0., 0., 0., 0., 7.]), {0: 15.0, 4: 15.0, 1: 0, 3: 0, 2: 0})

In [258]:
res=[]
p=[]
for j in range(4):
    p.append([all_alters_equivalent_path(network,geo,i,j)])
p

[[[0]], [[0, 4]], [[0, 3, 4]], [[0, 4]]]

In [225]:
network

Array([[0, 3, 0, 0, 1],
       [3, 0, 8, 0, 2],
       [0, 8, 0, 1, 0],
       [0, 0, 1, 0, 4],
       [1, 2, 0, 4, 0]], dtype=int64)

In [238]:
geo

Array([[0., 3., 6., 5., 1.],
       [3., 0., 7., 6., 2.],
       [6., 7., 0., 1., 5.],
       [5., 6., 1., 0., 4.],
       [1., 2., 5., 4., 0.]], dtype=float64, weak_type=True)

In [263]:
geo
# je sais que le chemin le plus court de 0 a 2 coute 3
# Y a t il un chemin au cout equivalent
# pour chaque alter je peux additioner le cout du chemin le plus court de 0 a l'alter + le chemin le plus court de l'alter a 2


Array([[0., 3., 6., 5., 1.],
       [3., 0., 7., 6., 2.],
       [6., 7., 0., 1., 5.],
       [5., 6., 1., 0., 4.],
       [1., 2., 5., 4., 0.]], dtype=float64, weak_type=True)

In [264]:
network

Array([[0, 3, 0, 0, 1],
       [3, 0, 8, 0, 2],
       [0, 8, 0, 1, 0],
       [0, 0, 1, 0, 4],
       [1, 2, 0, 4, 0]], dtype=int64)

In [290]:
short_path_ij

Array(6., dtype=float64, weak_type=True)

In [311]:
geo[alter,j]

Array(5., dtype=float64, weak_type=True)

In [312]:
geo[i,alter]

Array(1., dtype=float64, weak_type=True)

In [309]:
i=0
j=2
alter=4
r=[]
direct_path_ij=network[i,j]
short_path_ij=geo[i,j]
path_ij_by_k=geo[i,alter]+geo[alter,j]
# Compute predecessors
r = []
if direct_path_ij != 0 and direct_path_ij == short_path_ij:
    r.append(i)
if short_path_ij == path_ij_by_k:
    r.append(alter)
print(r)

[4]


In [313]:
import numpy as np

def compute_predecessors(geo, direct, i, j):
    """
    Compute the predecessors of node j from node i based on geodesic distances.

    Parameters:
        geo (np.ndarray): Geodesic distance matrix.
        direct (np.ndarray): Adjacency matrix representing direct connections.
        i (int): Source node index.
        j (int): Target node index.

    Returns:
        list: List of predecessors of node j.
    """
    r = []  # List of predecessors
    direct_path_ij = direct[i, j]
    short_path_ij = geo[i, j]

    # Check direct predecessor
    if direct_path_ij != 0 and direct_path_ij == short_path_ij:
        r.append(i)

    # Check intermediate predecessors
    for alter in range(geo.shape[0]):
        if alter != i and alter != j:  # Avoid self-loops and direct connections
            # Verify both segments are valid shortest paths
            if geo[i, alter] + geo[alter, j] == short_path_ij:
                if geo[i, alter] == geo[i, alter] and geo[alter, j] == geo[alter, j]:
                    r.append(alter)

    return r

# Test the function
i, j = 0, 2
predecessors = compute_predecessors(geo, network, i, j)
print(f"Predecessors of node {j} from node {i}: {predecessors}")


Predecessors of node 2 from node 0: [3, 4]


In [320]:
geo

Array([[0., 3., 6., 5., 1.],
       [3., 0., 7., 6., 2.],
       [6., 7., 0., 1., 5.],
       [5., 6., 1., 0., 4.],
       [1., 2., 5., 4., 0.]], dtype=float64, weak_type=True)

In [319]:
network

Array([[0, 3, 0, 0, 1],
       [3, 0, 8, 0, 2],
       [0, 8, 0, 1, 0],
       [0, 0, 1, 0, 4],
       [1, 2, 0, 4, 0]], dtype=int64)

In [323]:
short_path_ij

Array(6., dtype=float64, weak_type=True)

In [326]:
i

0

In [324]:
geo[i, alter] 

Array(5., dtype=float64, weak_type=True)

In [322]:
    r = []  # List of predecessors
    i, j = 0, 2
    direct_path_ij = direct[i, j]
    short_path_ij = geo[i, j]
    direct_path_ij != 0 and direct_path_ij == short_path_ij
    range(geo.shape[0])
    alter = 3
    geo[i, alter] + geo[alter, j] == short_path_ij



Array(True, dtype=bool, weak_type=True)

In [295]:
G = nx.from_numpy_array(network, create_using=nx.DiGraph)
S, P, sigma, D =nx.algorithms.centrality.betweenness._single_source_dijkstra_path_basic(G,0,'weight')
nx.algorithms.centrality.betweenness._single_source_dijkstra_path_basic(G,0,'weight')
#S=This list represents the shortest path tree
#P[w] contains the list of nodes from which w is reachable with the same shortest path distance
#sigma: This dictionary represents the number of distinct shortest paths from the source node s
# D: This dictionary holds the shortest path distance from the source s to each node.

([0, 4, 1, 3, 2],
 {0: [], 1: [0, 4], 2: [3], 3: [4], 4: [0]},
 {0: 2.0, 1: 4.0, 2: 2.0, 3: 2.0, 4: 2.0},
 {0: 0, 4: 1, 1: 3, 3: 5, 2: 6})

In [202]:
betweenness2=np.zeros(len(s))
nx.algorithms.centrality.betweenness._accumulate_basic(betweenness2,S, P, sigma,i)

(array([0. , 0. , 0. , 1. , 2.5]), {0: 4.0, 4: 2.5, 1: 0, 3: 1.0, 2: 0})

In [107]:
jnp.sum(t, axis=1)

Array([7., 4., 6., 4., 3.], dtype=float64)

# Dyadic measures
## Assortatitivity
## Transitivity

# Global measures

## Geodesic distance

In [14]:
import numpy as np
def networkx_geodesic_distance(adj_matrix):
    """
    Compute the geodesic distance using NetworkX for validation.
    Args:
        adj_matrix: 2D numpy array representing the weighted adjacency matrix of a graph.

    Returns:
        A 2D numpy array containing the shortest path distances between all pairs of nodes.
    """
    G = nx.from_numpy_array(adj_matrix, create_using=nx.DiGraph)
    dist = dict(nx.all_pairs_dijkstra_path_length(G))
    n_nodes = len(dist)
    dist_matrix = np.full((n_nodes, n_nodes), np.inf)

    for i in range(n_nodes):
        for j in dist[i]:
            dist_matrix[i, j] = dist[i][j]
    
    return dist_matrix

assert np.allclose(bi.net.geodesic_distance(jnp.array(network)), networkx_geodesic_distance(network), atol=1e-5), "Results do not match!"
print("Validation passed!")

c:\Users\SSosa\AppData\Local\Programs\Python\Python312\Lib\site-packages\jax\_src\ops\scatter.py:92: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float64 to dtype=int32 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(


Validation passed!


## Diameter

In [16]:
bi.net.geodesic_distance(jnp.array(network))

c:\Users\SSosa\AppData\Local\Programs\Python\Python312\Lib\site-packages\jax\_src\ops\scatter.py:92: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float64 to dtype=int32 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(


Array([[0., 3., 6., 5., 1.],
       [3., 0., 7., 6., 2.],
       [6., 7., 0., 1., 5.],
       [5., 6., 1., 0., 4.],
       [1., 2., 5., 4., 0.]], dtype=float64, weak_type=True)

In [ ]:
bi.net.diameter(jnp.array(network)) # this need to be the max value of the geodesic distance

c:\Users\SSosa\AppData\Local\Programs\Python\Python312\Lib\site-packages\jax\_src\ops\scatter.py:92: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float64 to dtype=int32 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(


Array(7., dtype=float64)

## Density

In [17]:
print("Network density (networkx):",bi.net.density(jnp.array(network)))
# Compute density using NetworkX
density_networkx = nx.density(G)
print("Network density (networkx):", density_networkx)

Network density (networkx): 0.6
Network density (networkx): 0.6


# Modularity

## Global clustering coefficient

## Global efficiency